In [ ]:
# --- 0. Orekit data ----------------------------------------------------------
import orekit, pathlib, os, json, numpy as np, pandas as pd, matplotlib.pyplot as plt
from java.io import File
from org.orekit.data import DataContext, DirectoryCrawler
orekit.initVM()

OREKIT_DIR = pathlib.Path("orekit-data").resolve()
dpm = DataContext.getDefault().getDataProvidersManager()
dpm.clearProviders()
dpm.addProvider(DirectoryCrawler(File(str(OREKIT_DIR))))
print("✔️ Orekit data registered")

# --- 1. Load measurements ----------------------------------------------------
df        = pd.read_parquet("GPS_clean.parquet")
meas_cols = json.load(open("meas_cols.json"))
meas      = df[meas_cols].values.astype("float64")   # (N, 6)
epochs    = df["time"].values.astype("datetime64[s]")

# --- 2. UKF + propagator -----------------------------------------------------
from aukf   import UnscentedKalman     # your Step-3 class
from utils  import OrbitPropagator     # wrapper around Orekit NumericalPropagator

ukf        = UnscentedKalman(OrbitPropagator(), meas_cols,
                             alpha=1e-3, beta=2.0, kappa=0.0,
                             q0=1e-2,  r0=25.0, adaptive=None)

ukf.init_from_measurement(0.0, meas[0])        # x₀ ← first epoch
prev_t = epochs[0]

hist = [ukf.x.copy()]
for t, z in zip(epochs[1:], meas[1:]):
    dt = (t - prev_t).astype(int)
    ukf.step(dt, z)
    hist.append(ukf.x.copy())
    prev_t = t

hist = np.array(hist)                           # (N, 6)

# --- 3. Quick ground-track plot ---------------------------------------------
plt.figure(figsize=(6,6))
plt.plot(hist[:,0]/1e3, hist[:,1]/1e3, lw=0.4)
plt.gca().set_aspect("equal")
plt.xlabel("X [km]"); plt.ylabel("Y [km]")
plt.title("UKF ground-track (ECEF)")
plt.tight_layout()
